<a href="https://colab.research.google.com/github/bitanb1999/SXC_Dissertation/blob/main/Zero_shot_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
# !wget https://raw.githubusercontent.com/bitanb1999/SXC_Dissertation/main/reddit_raw_data/dep_1000.csv
# !wget https://raw.githubusercontent.com/bitanb1999/SXC_Dissertation/main/twitter_data/2021-01-31-happy.csv
# !wget https://raw.githubusercontent.com/bitanb1999/SXC_Dissertation/main/reddit_raw_data/sui_1000.csv
# !wget https://raw.githubusercontent.com/bitanb1999/SXC_Dissertation/main/twitter_data/2021-01-31-dep.csv

In [79]:
red_df=pd.read_csv("/content/dep_1000.csv").drop(["Unnamed: 0","id","url","num_comments","created"],axis=1)
red_df.head()

,title,score,subreddit,body
0,"Regular check-in post, with information about ...",10,depression,Welcome to /r/depression's check-in post - a p...
1,Our most-broken and least-understood rules is ...,2366,depression,We understand that most people who reply immed...
2,I didn’t didn’t leave the room today.,74,depression,Got up once to use the bathroom but the rest w...
3,When you have everything but still don’t want ...,43,depression,"I have an amazing boyfriend, a dog, a beautifu..."
4,No motivation to do anything and feel like I'm...,363,depression,I know I (21F) should be looking for a job and...


In [80]:
red_df2=pd.read_csv("/content/sui_1000.csv").drop(["Unnamed: 0","id","url","num_comments","created"],axis=1)
red_df2.head()

,title,score,subreddit,body
0,New wiki on how to avoid accidentally encourag...,1778,SuicideWatch,We've been seeing a worrying increase in pro-s...
1,Please remember that NO ACTIVISM of any kind i...,713,SuicideWatch,"Activism, i.e. advocating or fundraising for s..."
2,im killing myself today,86,SuicideWatch,"As the title says, im killing myself today. ev..."
3,Another day another prayer hoping to die in my...,62,SuicideWatch,NaN
4,i pressed a loaded gun to my temple,16,SuicideWatch,(22) I am in the middle of the forest right no...


In [81]:
red_df=red_df[(red_df.score>40)]

In [82]:
red_df2=red_df2[(red_df2.score>40)]
df=pd.concat([red_df,red_df2])
df.head()

,title,score,subreddit,body
1,Our most-broken and least-understood rules is ...,2366,depression,We understand that most people who reply immed...
2,I didn’t didn’t leave the room today.,74,depression,Got up once to use the bathroom but the rest w...
3,When you have everything but still don’t want ...,43,depression,"I have an amazing boyfriend, a dog, a beautifu..."
4,No motivation to do anything and feel like I'm...,363,depression,I know I (21F) should be looking for a job and...
5,Loneliness kills...,56,depression,Loneliness kills your soul. I've been divorced...


In [83]:
df["body"]=df["title"]+" "+df["body"]
df=df.drop("score",axis=1).rename(columns={"subreddit":"label"})
df.head()

,title,label,body
1,Our most-broken and least-understood rules is ...,depression,Our most-broken and least-understood rules is ...
2,I didn’t didn’t leave the room today.,depression,I didn’t didn’t leave the room today. Got up o...
3,When you have everything but still don’t want ...,depression,When you have everything but still don’t want ...
4,No motivation to do anything and feel like I'm...,depression,No motivation to do anything and feel like I'm...
5,Loneliness kills...,depression,Loneliness kills... Loneliness kills your soul...


In [84]:
df=df.drop("title",axis=1)

In [85]:
twit_df=pd.read_csv("/content/2021-01-31-dep.csv").drop(["Date Created","Source of Tweet","Number of Likes"],axis=1)[:1000]
twit_df.head()

,Tweet
0,“When depression tempts us to question our pur...
1,@joeyrettinger You know what else makes a depr...
2,Never mind the depression is coming back I wan...
3,(I know Devin was kind if already in a depress...
4,@HerbGamer More like how it got to civil war. ...


In [86]:
twit_df2=pd.read_csv("/content/2021-01-31-happy.csv").drop(["Date Created","Source of Tweet","Number of Likes"],axis=1)[:1000]
twit_df2.head()

,Tweet
0,Go tell him happy birthday boys ❤
1,Happy sunday ❤
2,@vnteggukie Don't skip ur breakfast and happy ...
3,Such passive aggressive piano playing I have n...
4,HAPPY 1 YEAR ANIVERSARY OF WALLS!!!! @Louis_To...


In [87]:

#clean data using helper code
import re
def transform(summary):
    # Lower case
    summary = summary.lower()
    summary=summary.replace('\n',' ')
    # Remove HTML
    summary = re.sub('<br /><br />', ' ', summary)
    
    # Replace all non-alphanumeric characters with spaces
    summary = re.sub(r'[^a-zA-Z0-9\s]', '', summary)
    
    return summary

In [88]:
twit_df["Tweet"]=twit_df["Tweet"].apply(lambda x: transform(x))
twit_df2["Tweet"]=twit_df2["Tweet"].apply(lambda x: transform(x))
twit_df["label"]=["depressed"]*twit_df.shape[0]
twit_df2["label"]=["not depressed"]*twit_df2.shape[0]

In [89]:
twit_df.head()

,Tweet,label
0,when depression tempts us to question our purp...,depressed
1,joeyrettinger you know what else makes a depre...,depressed
2,never mind the depression is coming back i wan...,depressed
3,i know devin was kind if already in a depressi...,depressed
4,herbgamer more like how it got to civil war mi...,depressed


In [90]:
twit_df2.head()

,Tweet,label
0,go tell him happy birthday boys,not depressed
1,happy sunday,not depressed
2,vnteggukie dont skip ur breakfast and happy we...,not depressed
3,such passive aggressive piano playing i have n...,not depressed
4,happy 1 year aniversary of walls louistomlinso...,not depressed


In [91]:
twit_df=pd.concat([twit_df,twit_df2])
twit_df

,Tweet,label
0,when depression tempts us to question our purp...,depressed
1,joeyrettinger you know what else makes a depre...,depressed
2,never mind the depression is coming back i wan...,depressed
3,i know devin was kind if already in a depressi...,depressed
4,herbgamer more like how it got to civil war mi...,depressed
...,...,...
995,requiiems omg youre not dutch and watch it i l...,not depressed
996,mattmoodym happy birthday httpstcolzuz7yy298,not depressed
997,just saying i backed leno since he played at l...,not depressed
998,this happy weight a come wid a happy gut,not depressed


In [92]:
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
!pip3 install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
 
    # decide sentiment as positive, negative and neutral
    return sentiment_dict["compound"]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
twit_df["sc"]=twit_df["Tweet"].apply(lambda x:sentiment_scores(x))
twit_df

,Tweet,label,sc
0,when depression tempts us to question our purp...,depressed,-0.7003
1,joeyrettinger you know what else makes a depre...,depressed,-0.5719
2,never mind the depression is coming back i wan...,depressed,-0.0263
3,i know devin was kind if already in a depressi...,depressed,-0.0387
4,herbgamer more like how it got to civil war mi...,depressed,-0.8081
...,...,...,...
995,requiiems omg youre not dutch and watch it i l...,not depressed,0.8733
996,mattmoodym happy birthday httpstcolzuz7yy298,not depressed,0.5719
997,just saying i backed leno since he played at l...,not depressed,0.9144
998,this happy weight a come wid a happy gut,not depressed,0.8126


In [94]:
twit_df=twit_df[((twit_df.label=="depressed") & (twit_df.sc<-0.40))|((twit_df.label=="not depressed") & (twit_df.sc>0.40))]

In [95]:
df=df.dropna(axis=0)
df["body"]=df["body"].apply(lambda x:transform(x))
df.head()

<ipython-input-95-c767051c1f8d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["body"]=df["body"].apply(lambda x:transform(x))


,label,body
1,depression,our mostbroken and leastunderstood rules is he...
2,depression,i didnt didnt leave the room today got up once...
3,depression,when you have everything but still dont want t...
4,depression,no motivation to do anything and feel like im ...
5,depression,loneliness kills loneliness kills your soul iv...


In [96]:
df["sc"]=df["body"].apply(lambda x:sentiment_scores(x))
df.head()

,label,body,sc
1,depression,our mostbroken and leastunderstood rules is he...,0.9973
2,depression,i didnt didnt leave the room today got up once...,0.8498
3,depression,when you have everything but still dont want t...,0.9175
4,depression,no motivation to do anything and feel like im ...,-0.5472
5,depression,loneliness kills loneliness kills your soul iv...,-0.9897


In [97]:
df=df[df.sc<-0.40]
df["label"]=["depressed"]*df.shape[0]

<ipython-input-97-3804ca627afc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"]=["depressed"]*df.shape[0]


In [98]:
df=df.rename(columns={"body":"Tweet"}).drop("sc",axis=1).sample(frac = 1).reset_index(drop=True)

In [99]:
twit_df=twit_df.drop("sc",axis=1).sample(frac=1).reset_index(drop=True)

In [100]:
df=pd.concat([twit_df,df]).sample(frac=1).reset_index(drop=True)
df.head()

,Tweet,label
0,sangwoo62629774 do the people around him reali...,depressed
1,shhouggg happy birthday love,not depressed
2,statedependent responses to intracranial brain...,depressed
3,my depression httpstcovtpepdceaq,depressed
4,wasting money on bad food is worst than depres...,depressed


In [101]:
df.to_csv("test_data")

## Zero Shot Classification:

In [31]:
test_data=pd.read_csv("/content/test_data").drop("Unnamed: 0",axis=1)
test_data.head()

,Tweet,label
0,sangwoo62629774 do the people around him reali...,depressed
1,shhouggg happy birthday love,not depressed
2,statedependent responses to intracranial brain...,depressed
3,my depression httpstcovtpepdceaq,depressed
4,wasting money on bad food is worst than depres...,depressed


In [32]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Your token has been saved in your configured git cred

In [33]:
#!pip install transformers
from transformers import TextClassificationPipeline,BertConfig,BertForSequenceClassification,BertTokenizerFast
config = BertConfig()
model = BertForSequenceClassification.from_pretrained('mental/mental-bert-base-uncased',config=config)
tokenizer = BertTokenizerFast.from_pretrained('mental/mental-bert-base-uncased', truncation=True,padding=True)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,truncation=True)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]
val=pipe("I feel lonely")[0]

Some weights of the model checkpoint at mental/mental-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental

In [34]:
val[0]

{'label': 'LABEL_0', 'score': 0.5476395487785339}

In [35]:
sorted(val, key=lambda d: d['score'])[0]['label']

'LABEL_1'

In [36]:
test_data["pred"]=test_data["Tweet"].apply(lambda x:sorted(pipe(x)[0], key=lambda d: d['score'])[0]['label'])

In [37]:
test_data["pred"]=["depressed" if i=="LABEL_0" else "not depressed" for i in test_data.pred]

In [40]:
len(test_data[test_data.label==test_data.pred])/len(test_data["label"])

0.5516129032258065

In [44]:
import transformers
tokenizer_name = 'gpt2'
model_name = 'gpt2'
tokenizer = transformers.GPT2TokenizerFast.from_pretrained(tokenizer_name)
configuration = transformers.GPT2Config()
model= transformers.GPT2ForSequenceClassification(configuration).from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,truncation=True)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]
val=pipe("I feel lonely")[0]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [47]:
test_data["gpt_pred"]=test_data["Tweet"].apply(lambda x:sorted(pipe(x)[0], key=lambda d: d['score'])[0]['label'])

In [50]:
test_data["gpt_pred"]=["depressed" if i=="LABEL_0" else "not depressed" for i in test_data.gpt_pred]

In [51]:
len(test_data[test_data.label==test_data.gpt_pred])/len(test_data["label"])

0.567741935483871

### Model Visualisations

In [52]:
!pip install bertviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 68.4 MB/s eta 0:00:00


In [57]:
from bertviz import head_view,model_view
from transformers import BertTokenizer, BertModel

In [54]:
# Load model and retrieve attention
model_version = 'mental/mental-bert-base-uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
sentence_a = "I do not want to live"
sentence_b = "I do not like to live"
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

Some weights of the model checkpoint at mental/mental-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert

In [55]:
head_view(attention,tokens)

<IPython.core.display.Javascript object>

In [58]:
model_view(attention,tokens)

<IPython.core.display.Javascript object>